# <b>ENGR 418 Project Stage 1</b>

<b>Group 7 <br>
Group Members:</b> <br>
Alif Aiman Ahmad Zukiman | 35338490 <br>
Anas Chellabi | 

_October 30, 2023_

## Problem Desription
We are to classify a Lego dataset into four different classes: 2x4 rectangles, 2x2 squares, 2x2 circles, and 2x1 rectangles. The classified Legos are then sorted according to their predicted classes. This will be done via the multi-class classification algorithm, as presented below.

### 1. Extracting Data

Import necessary libraries:

In [3]:
import os                                                       # for recognising path directories
import numpy as np  
from numpy import asarray
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression             # algorithm to be used
from sklearn.metrics import confusion_matrix, accuracy_score    # accuracy testing
from PIL import Image                                           # image processor

Assign arrays of training and testing by redirecting the arrays to the specified paths:

In [12]:
n_classes = 4   # no. of classes

# create empty lists for four classes
folder_training = [0 for i in range(n_classes)]
folder_testing = [0 for i in range(n_classes)]
classes = [i for i in range(n_classes)]

training_path = "Lego_dataset_1\\training"
testing_path = "Lego"

# NEED FIX: the training folder does not divide into different classes
# assign each folder rows into paths of each class
for i in range(4):
    folder_training[i] = "Lego_dataset_1/training/" + str(i) + '/'
    folder_testing[i] = "Lego_dataset_1/testing/" + str(i) + '/'



Define a function "get_data" to retrieve information about the chosen images:

In [13]:
def get_data(folder, im_width, label, n_samples):
    
    file_names = os.listdir(folder)
    # print(file_names[0])

    # x,y coordinates for storing the image pixels
    x = np.empty((n_samples, im_width**2))
    y = np.empty((n_samples, 1))

    # extract image information into x,y
    for i in range(n_samples):
        path = folder + file_names[i]
        im = Image.open(path).convert('L')      # open image and convert to grayscale
        im = im.resize((im_width,im_width))     # resize to pixel size desired
        im_array = asarray(im)
        x[i,:] = im_array.reshape(1,-1)
        y[i,0] = classes[label]

    return x,y

### 2. Parameter Initialization and Training Data Preparation

In [37]:
a = os.listdir("E:\\EDU UBC\\Assignments\\ENGR 418 (MCHL)\\Lego_dataset_1\\training\\")
print(type(a[0]))

count = 0
for i in a:
    if "2b1" in i:
        count += 1

print(count)

<class 'str'>
36


In [16]:
n_classes = len(classes) # no. of classes
im_width = 64   # pixel size to be considered



FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Lego_dataset_1/training/0/'